In [0]:
-- =========================================================
-- 05_gold_rules.sql  (Día 7 – Procesamiento Gold)
-- Objetivo:
--  - Crear schemas ref/gold si no existen
--  - Enriquecer desde Silver con reglas y scoring
--  - Crear gold.transactions_enriched y gold.transactions_alerts
-- Reglas:
--   R1: amount > 15000                  (peso 40)
--   R2: hour < 6 OR hour > 22           (peso 20)
--   R3: último mes > (μ + 2σ) tx/mes    (peso 50)
-- =========================================================

-- 0) Contexto UC
USE CATALOG finance_catalog;
CREATE SCHEMA IF NOT EXISTS ref;
CREATE SCHEMA IF NOT EXISTS gold;

-- Ref placeholder
CREATE TABLE IF NOT EXISTS ref.placeholders (k STRING, v STRING)
TBLPROPERTIES('quality'='ref');


In [0]:
-- 1) Base desde Silver (limpia) + hora + fuera de horario
DROP VIEW IF EXISTS v_tx_silver;
CREATE OR REPLACE TEMP VIEW v_tx_silver AS
SELECT
  t.*,
  hour(t.timestamp) AS tx_hour,
  CASE WHEN hour(t.timestamp) < 6 OR hour(t.timestamp) > 22 THEN 1 ELSE 0 END AS rule_off_hours
FROM finance_catalog.silver.transactions t;

In [0]:
-- 2) Estadística mensual por cliente (conteo)
DROP VIEW IF EXISTS v_monthly_counts;
CREATE OR REPLACE TEMP VIEW v_monthly_counts AS
SELECT
  customer_id,
  date_trunc('month', timestamp) AS ym,
  COUNT(*)                       AS tx_count
FROM v_tx_silver
GROUP BY customer_id, date_trunc('month', timestamp);

In [0]:
-- 3) μ y σ por cliente + último mes del dataset
DROP VIEW IF EXISTS v_stats_per_customer;
CREATE OR REPLACE TEMP VIEW v_stats_per_customer AS
SELECT
  customer_id,
  AVG(tx_count)         AS mu,
  stddev_samp(tx_count) AS sigma
FROM v_monthly_counts
GROUP BY customer_id;

DROP VIEW IF EXISTS v_last_month;
CREATE OR REPLACE TEMP VIEW v_last_month AS
SELECT MAX(ym) AS last_month FROM v_monthly_counts;


In [0]:
-- 4) Clientes con “var-spike” en el último mes (tx_count > μ + 2σ)
DROP VIEW IF EXISTS v_var_spike_customers;
CREATE OR REPLACE TEMP VIEW v_var_spike_customers AS
SELECT
  mc.customer_id,
  mc.ym,
  mc.tx_count,
  spc.mu,
  spc.sigma,
  (spc.mu + 2 * COALESCE(spc.sigma, 0)) AS threshold
FROM v_monthly_counts mc
JOIN v_stats_per_customer spc USING (customer_id)
CROSS JOIN v_last_month lm
WHERE mc.ym = lm.last_month
  AND mc.tx_count > (spc.mu + 2 * COALESCE(spc.sigma, 0));


In [0]:
-- 5) Enriquecer + scoring
-- Pesos: R1=40, R2=20, R3=50
CREATE OR REPLACE TABLE finance_catalog.gold.transactions_enriched
TBLPROPERTIES('quality'='gold') AS
WITH base AS (
  SELECT
    s.*,                                         -- trae tx_hour y rule_off_hours de v_tx_silver
    CASE WHEN s.amount > 15000 THEN 1 ELSE 0 END AS rule_high_amount
  FROM v_tx_silver s
)
SELECT
  b.*,
  CASE
    WHEN vs.customer_id IS NOT NULL
         AND date_trunc('month', b.timestamp) = vs.ym
    THEN 1 ELSE 0
  END AS rule_var_spike,
  -- Risk score
  ( (CASE WHEN b.amount > 15000 THEN 40 ELSE 0 END)
  + (b.rule_off_hours * 20)
  + (CASE
       WHEN vs.customer_id IS NOT NULL
            AND date_trunc('month', b.timestamp) = vs.ym
       THEN 50 ELSE 0
     END)
  ) AS risk_score,
  -- Flag de sospechosa si al menos una regla pega
  CASE
    WHEN ( (CASE WHEN b.amount > 15000 THEN 1 ELSE 0 END)
         + b.rule_off_hours
         + (CASE
              WHEN vs.customer_id IS NOT NULL
                   AND date_trunc('month', b.timestamp) = vs.ym
              THEN 1 ELSE 0
            END)
        ) >= 1
    THEN TRUE ELSE FALSE
  END AS is_suspicious
FROM base b
LEFT JOIN v_var_spike_customers vs
  ON b.customer_id = vs.customer_id;



num_affected_rows,num_inserted_rows


In [0]:
-- 6) Tabla de alertas
CREATE OR REPLACE TABLE finance_catalog.gold.transactions_alerts
TBLPROPERTIES('quality'='gold') AS
SELECT *
FROM finance_catalog.gold.transactions_enriched
WHERE is_suspicious = TRUE;

num_affected_rows,num_inserted_rows


In [0]:
-- 7) Validaciones rápidas
SELECT COUNT(*) AS alerts_count
FROM finance_catalog.gold.transactions_alerts;

SELECT transaction_id, customer_id, amount, tx_hour, risk_score,
       rule_high_amount, rule_off_hours, rule_var_spike
FROM finance_catalog.gold.transactions_alerts
ORDER BY risk_score DESC
LIMIT 10;

SELECT date(timestamp) AS day, COUNT(*) AS alerts_per_day
FROM finance_catalog.gold.transactions_alerts
GROUP BY 1
ORDER BY 1;

day,alerts_per_day
2025-01-01,34
2025-01-02,20
2025-01-03,38
2025-01-04,22
2025-01-05,40
2025-01-06,24
2025-01-07,24
2025-01-08,46
2025-01-09,44
2025-01-10,50
